We create a "grid" from vectors u1 and u2--i.e. the graph whose flow we are concerned about. We then can print a visual display of whether each of these points lie in the circle or square, in order to gain intuition regarding the possibility of flow using only 9 directions.

The main question we want to answer is: What is the smallest $n$ such that there are $\geq 4n+2$ points of the circle and $\geq 4n+2$ points of the square in $n \times n$ region (we refer to this region as a "cube").
To gain intuition about this question, we then create a 10,000 by 10,000 grid and set $n$ equal to integer values greater than 4. We print every unique $n \times n$ region there is (to gain intuition), and find the number of points in the circle and square for every $n \times n$ region. 

We also find the maximum difference between the number of points in the square and circle across all $n \times n$ regions. 

In [1]:
###############################
import math
import time
import random
from collections import deque
import matplotlib.pyplot as plt
import numpy as np
###############################

In [2]:
###############################
# Global variables
###############################
c = (0.5, 0.5)                       # coordinate for center of original shapes
r = 0.49                             # radius for circle
s = math.sqrt(math.pi*r**2)          # side length for square
#g = (1,0)                           # direction, choose out of S = {(1,0), (0,1), (1,1)}
#u1 = (1/math.sqrt(2),1/math.sqrt(5)) # vector 1
#u2 = (1/math.sqrt(3),1/math.sqrt(6)) # vector 2
#u3 = (1/math.sqrt(2)+1/math.sqrt(5),1/math.sqrt(3)+1/math.sqrt(6)) # vector 3
GRID_SIZE = 2000                       # 10,000


def checkValid():
    if (r > 0.5 or s > 1):
        raise Exception("Shapes are too big!")

In [3]:
#########################################################
# Helper functions for finding out if point is in shape
#########################################################

# Helper function to determine if a given point is inside square
# Similar to Quantifier Elimination file: new_square function 
# Input: point--tuple representing a given point; v--tuple representing vector to shift original square by
# Output: 0 if point outside square, 1 if point inside/on boundary of square

def pt_in_square(point, v=(0,0)):
    
    center = ((c[0]+v[0])%1, (c[1]+v[1])%1) #center of new square
    x1 = (center[0]-s/2)%1
    x2 = (center[0]+s/2)%1
    y1 = (center[1]-s/2)%1
    y2 = (center[1]+s/2)%1
    
    
    if x1 <= x2 and y1 <= y2: #rectangle does no overflow mod 1
        if x1 <= point[0] and point[0] <= x2 and y1 <= point[1] and point[1] <= y2:
            return 1

    elif x1 > x2 and y1 <= y2: #rectangle overflows horizontally
        if x1 <= point[0] or point[0] <= x2 and y1 <= point[1] and point[1] <= y2:
            return 1
        
    elif x1 <= x2 and y1 > y2: #rectangle overflows vertically
        if x1 < point[0] and point[0] < x2 and y1 < point[1] or point[1] < y2:
            return 1
            
    else: #rectangle overflows both horizontally and vertically (occupies corners of the torus)
        if x1 < point[0] or point[0] < x2 and y1 < point[1] or point[1] < y2:
            return 1
            
    return 0


# Helper function to determine if a given point is inside circle
# Similar to Quantifier Elimination file: new_circle function
# Input: point--tuple representing a point; v--tuple representing vector to shift original circle by
# Output: 0 if point outside circle, 1 if point inside/on boundary of circle

def pt_in_circle(point, v=(0,0)):
    
    center = ((c[0]+v[0])%1, (c[1]+v[1])%1) #center of new circle
    x1 = (center[0]-r)%1
    x2 = (center[0]+r)%1
    y1 = (center[1]-r)%1
    y2 = (center[1]+r)%1
    
    if x1 < x2 and y1 < y2: #circle does not overflow mod 1
        if (point[0]-center[0])**2 + (point[1]-center[1])**2 <= r**2:
            return 1
    
    if x1 > x2: #circle overflows horizontally (and maybe vertically)
        if (point[0]-center[0])**2 + (point[1]-center[1])**2 <= r**2:
            return 1
        elif center[0] > 0.5 and (point[0]+1-center[0])**2 + (point[1]-center[1])**2 <= r**2:
            return 1
        elif center[0] < 0.5 and (point[0]-1-center[0])**2 + (point[1]-center[1])**2 <= r**2:
            return 1
        
    if y1 > y2: #circle overflows vertically (and maybe horizontally)
        if (point[0]-center[0])**2 + (point[1]-center[1])**2 <= r**2:
            return 1
        elif center[1] > 0.5 and (point[0]-center[0])**2 + (point[1]+1-center[1])**2 <= r**2:
            return 1
        elif center[1] < 0.5 and (point[0]-center[0])**2 + (point[1]-1-center[1])**2 <= r**2:
            return 1
    
    if x1 > x2 and y1 > y2: #circle overflows both horizontally and vertically--check opposite corner
        if center[0] < 0.5 and center[1] < 0.5: # bottom left
            if (point[0]-1-center[0])**2 + (point[1]-1-center[1])**2 <= r**2:
                return 1
        elif center[0] < 0.5: # top left
            if (point[0]-1-center[0])**2 + (point[1]+1-center[1])**2 <= r**2:
                return 1
        elif center[1] < 0.5: # bottom right
            if (point[0]+1-center[0])**2 + (point[1]-1-center[1])**2 <= r**2:
                return 1
        else: # top right
            if (point[0]+1-center[0])**2 + (point[1]+1-center[1])**2 <= r**2:
                return 1
    
    return 0
    
    

In [4]:
#########################################################
# Helper function that creates 2D grid to "draw" on
#########################################################

# Generate finite set F = {i*u1 + j*u2 + pt: 0 <= i,j < N} in the form of a 2D array. 
# Uses u1, u2 global variables (tuples) # Not anymore!
# Similar to Discrepancy file: make_f function
# Input: pt starting point (tuple), N upper bound on i,j
# Output: array F where F[i][j] = (i*u1 + j*u2 + pt)%1, as a tuple

def make_set(pt, N, u1, u2, print_set=False):
    F = list()
    x1 = u1[0]
    y1 = u1[1]
    x2 = u2[0]
    y2 = u2[1]
    
    for i in range(N):
        row_list = list()
        for j in range(N):
            x = (i*x1+j*x2+pt[0])%1
            y = (i*y1+j*y2+pt[1])%1
            row_list.append((x,y))
        F.append(row_list)
        if print_set:
            print(row_list)
        
    return F


In [5]:
#########################################################
# Function that makes 2D grid with *,+,-,0
#########################################################

# Make n by n grid of points formed by vector v.
# * for pts in both circle and square, + square, - circle, 0 neither
# Input: v--vector to shift by (tuple), pt--starting point (tuple), n--size of grid (int)
def make_grid(v, pt, n, u1, u2, print_grid=False,print_set=False):
    
    if print_set:
        F = make_set(pt,n,u1,u2,print_set=True)
    else:
        F = make_set(pt,n,u1,u2)
    grid = list()
    
    for i in range(n):
        row_grid = list()
        for j in range(n):
            point = F[i][j]
            in_sq = pt_in_square(point,v)
            in_circ = pt_in_circle(point,v)
            
            if in_sq and in_circ:
                row_grid.append('*')
            elif in_sq:
                row_grid.append('+')
            elif in_circ:
                row_grid.append('-')
            else:
                row_grid.append('0')
            
            if print_grid:
                print(row_grid[-1], end=" ")
        
        if print_grid:
            print()
        grid.append(row_grid)
        
    return grid


In [6]:
#########################################################
# Helper functions that checks for "fenced in" pattern
# 0 0 0
# 0 + 0
# 0 0 0
# (and also the '-' counterpart)
#########################################################


# Helper function for checking neighbors of single point
# Input: i,j coordinates of grid to check; n=size of grid; grid=grid with *+-0
# Output: 0 if not all neighbors are zeros, 1 if all neighbors are zeros
def all_zeros(i, j, n, grid):
    for k in range(-1,2,1):
        for m in range(-1,2,1):
            if i+k >= 0 and i+k<n and j+m >= 0 and j+m<n:
                if grid[i+k][j+m] != '0':
                    return 0
    return 1
            
    
    
# Function that checks all neighbors of all + and - 
# Output: 1 if "fenced in" pattern found; 0 otherwise
def check_neighbors(v, pt, n, print_grid=False):
    grid = make_grid(v, pt, n, print_grid)
    for i in range(n):
        for j in range(n):
            if (grid[i][j] == '+' or grid[i][j] == '-') and all_zeros(i,j,n,grid):
                print("Fenced pattern found at " + str(i) + ", " + str(j))
                return 1 
    return 0


In [7]:
##################################################
# Find smallest n
#  such that for every n by n region,
#   the number of points in circle >= 4n+2
#   and the number of points in square >= 4n+2
##################################################

# Helper function that counts number of points in circle and square for given n by n region of grid
# Output: tuple (num_pts_circ, num_pts_sq)
def count_pts_circ_sq(n, grid, i, j):
    num_circ = 0
    num_sq = 0
    for k in range(i,i+n):
        for m in range(j,j+n):
            if grid[k][m] == '-':
                num_circ += 1
            elif grid[k][m] == '+':
                num_sq += 1
            elif grid[k][m] == '*':
                num_circ += 1
                num_sq += 1
    return (num_circ, num_sq)


# Returns the minimum number of points in circle/square in all n by n regions in the grid
def find_min_pts(n, grid):    
    min_pts = GRID_SIZE**2
    for i in range(GRID_SIZE-n+1):
        for j in range(GRID_SIZE-n+1):
            num_circ, num_sq = count_pts_circ_sq(n,grid,i,j)
            if num_circ < min_pts:
                min_pts = num_circ
            if num_sq < min_pts:
                min_pts = num_sq
    return min_pts


# Returns smallest value of n, or -1 if no n < 10 found
def find_n(grid):
    for i in range(4,10):
        if find_min_pts(i,grid) >= 4*i+2:
            return i
    return -1
    

In [8]:
#################################################
# Find max difference between number points 
#  in circle and square for each n by n region
#################################################

# Returns tuple: (max difference, coordinate of upper left corner at which max dif occurs at) 
def find_max_dif(n, grid):
    max_dif = 0
    max_x = -1
    max_y = -1
    for i in range(GRID_SIZE-n+1):
        for j in range(GRID_SIZE-n+1):
            num_circ, num_sq = count_pts_circ_sq(n,grid,i,j)
            if abs(num_circ-num_sq) > max_dif:
                max_dif = abs(num_circ-num_sq)
                max_x = i
                max_y = j
    return max_dif, (max_x, max_y)


In [9]:
####################################
# Testing for optimal u1, u2
####################################


# Helper function that returns list of integers between a and b that aren't perfect squares
def make_list_nums(a,b):
    nums = list()
    for i in range(a,b+1):
        if math.sqrt(i)%1 != 0:
            nums.append(i)
    return nums
    

# Function that loops through many algebraic irrationals as possibilities for u1 and u2
# Returns the u1, u2 with the smallest max_dif, i.e. difference between number of pts in circle and square
# Assumes n = 7 (i.e. 7 by 7 regions)
# Input: a,b start and end integer for values of u1, u2 to try; i.e. a=1, b=3 would try 1/sqrt(2), 1/sqrt(3) in all
#  possible combinations for u1, u2
# Output: two tuples u1 and u2; the value of max_dif

def find_best_u(a,b,v=(0,0), pt=(0,0)):
    nums = make_list_nums(a,b)
    min_dif = GRID_SIZE
    min_u1 = (0,0)
    min_u2 = (0,0)
    for i in nums:
        for j in nums:
            for k in nums:
                for l in nums:                 #you are double counting. u1=(i,j),u2=(k,l) vs u1=(k,l),u2=(i,j)
                    if i==k and j==l:
                        break
                    u1 = (1/math.sqrt(i),1/math.sqrt(j))
                    u2 = (1/math.sqrt(k),1/math.sqrt(l))
                    grid = make_grid(v, pt, GRID_SIZE, u1, u2)
                    max_dif, max_coord = find_max_dif(7,grid)
                    
                    print(str(u1) + ", " + str(u2))
                    print(str(max_dif))
                    
                    if max_dif < min_dif:
                        min_dif = max_dif
                        min_u1 = "(1/math.sqrt("+str(i)+"),1/math.sqrt("+str(j)+"))"
                        min_u2 = "(1/math.sqrt("+str(k)+"),1/math.sqrt("+str(l)+"))"
                        
    return min_u1, min_u2, min_dif                
    

    

In [10]:
####################################
# Testing for optimal u1, u2
#  (with uniform intervals)
####################################

#"Euclid Algorithm"
def gcd(x,y):
    #print("gcd of "+str(x)+", "+str(y))
    while(y):
        x,y=y,x%y
    #print(x)
    return x


# Helper function: returns true if tuples a and b are relatively prime
# Used to make sure test u1, u2 are independent
def rel_prime(a,b):
    if gcd(a[0],b[0]) == 1 and gcd(a[1],b[1]) == 1:
        return True
    else:
        return False
    
    
# Like the find_best_u function, but instead of algebraic irrationals, just try intervals of 0.1
# This doesn't account for linearly independent u1 and u2 yet. TODO: fix that
def find_best_u_uniform(step = 10, v=(0,0), pt=(0,0)):
    nums = list()
    for i in range(step):
        nums.append(i/step)
    min_dif = GRID_SIZE
    min_u1 = (0,0)
    min_u2 = (0,0)
    for i in nums:
        for j in nums:
            if i == 0 and j == 0:
                break
            for k in nums:
                for l in nums:
                    if k == 0 and l == 0:
                        break
                    if i==k and j==l:
                        break
                    if not rel_prime((i*step,j*step),(k*step,l*step)):
                        break
                    u1 = (i,j)
                    u2 = (k,l)
                    grid = make_grid(v, pt, GRID_SIZE, u1, u2)
                    max_dif, max_coord = find_max_dif(7,grid)
                    
                    print(str(u1) + ", " + str(u2))
                    print(str(max_dif))
                    
                    if max_dif < min_dif:
                        min_dif = max_dif
                        min_u1 = "("+str(i)+","+str(j)+")"
                        min_u2 = "("+str(k)+","+str(l)+")"
                        
    return min_u1, min_u2, min_dif   



In [11]:
####################################
# main
####################################

def main():
    #test()
    
    v = (0,0) 
    pt = (0,0) 
    
    #test_aug_path()
    print("main")
    print(GRID_SIZE)
    
    
    

    
if __name__ == "__main__":
    main()

main
2000


So list pop(0) has O(n) time complexity. Use collections deque instead
https://www.geeksforgeeks.org/deque-in-python/


Notes:
    find_min_pts(n,grid) seems to return 22 for n=6 for grid_size = 59,100,1000
    find_min_pts(n,grid) seems to return 31 for n=7 for grid_size = 59,100,1000

Reading up on augmenting paths algorithms

Main concerns:
    - how to encode this 2d array as a graph efficiently?
    - what exactly is "augmented path"
    - will this code take forever to run?
    
https://www.hackerearth.com/practice/algorithms/graphs/maximum-flow/tutorial/#:~:text=An%20augmenting%20path%20is%20a,then%20the%20flow%20is%20maximum.
- Apparently Ford Fulkerson alg has its weaknesses, although they may not be entirely applicable to our case
- Dinic's alg is another possibility, and is supposedly faster (not sure if it is necessarily faster for our case)




In [12]:
def holding_bay():
    v = (0,0) # uses original square and circle
    pt = (0,0) # starting point for our 'grid' F = {i*u1 + j*u2 + pt: 0 <= i,j < N}
    #n = 7 # size of cube regions
    #print("Check neighbors returns (0 = no violations found): " + str(check_neighbors(v,pt,GRID_SIZE)))
    
    grid = make_grid(v, pt, GRID_SIZE)
    tic = time.perf_counter()
    n = find_n(grid)
    toc = time.perf_counter()
    print("n = " + str(n) + ", time elapsed: " + str(toc-tic) + " sec")
    if n == -1:
        raise Exception("n = -1")
    tic = time.perf_counter()
    #"""
    max_dif, max_coord = find_max_dif(n,grid)
    toc = time.perf_counter()
    print("max_dif = " + str(max_dif) 
          + ", occurs at the n by n region starting at " + str(max_coord))
    print("Time elapsed: " + str(toc-tic) + " sec")

    
    """
    # From square to circle. Starting at grid_sq[i][j]
    def follow_unmatched_edges(i,j):
        global grid_sq, grid_circ, edges_sq_to_circ, edges_circ_to_sq, matched_edges_sq, matched_edges_circ
        for d in edges_sq_to_circ[i][j]:
            if grid_circ[i+d[0]][j+d[1]] == IND_YES:
                grid_sq[i][j] == IND_MATCHED
                grid_circ[i+d[0]][j+d[1]] == IND_MATCHED
                matched_edges_sq[i][j] = d
                matched_edges_circ[i+d[0]][j+d[1]] == (-d[0], -d[1])
                return True
        for d in edges_sq_to_circ[i][j]:
            
    
            
    # Run augmenting path algorithm
    # Note: this will edit grid_sq and grid_circ
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            
            
            if grid_sq[i][j] == IND_YES:
                for d in edges_sq_to_circ[i][j]:
                    if grid_circ[i+d[0]][j+d[1]] == IND_YES:
                        grid_sq[i][j] == IND_MATCHED
                        grid_circ[i+d[0]][j+d[1]] == IND_MATCHED
                        matched_edges_sq[i][j] = d
                        matched_edges_circ[i+d[0]][j+d[1]] == (-d[0], -d[1])
                        
                if grid_sq[i][j] == IND_YES: # all circs it matches to are already matched
                    
    """
    """
    #u1 = (1/math.sqrt(2),1/math.sqrt(2)) 
    #u2 = (1/math.sqrt(3),1/math.sqrt(5))
    #u1 = (3/math.sqrt(101),1/math.sqrt(101)) #close to (0.3,0.1)
    #u2 = (1/math.sqrt(101),7/math.sqrt(101)) #close to (0.1,0.7)
    #u1=(0.3,0.1)
    #u2=(0.1,0.7)
    #n = find_n(grid)
    #print("n = " + str(n))
    max_dif, max_coord = find_max_dif(7,grid)
    print("max_dif = " + str(max_dif) + ", at " + str(max_coord))
    """
    
    """
    tic = time.perf_counter()
    u1, u2, max_dif = find_best_u_uniform()
    toc = time.perf_counter()
    print("u1 = " + str(u1) + ", u2 = " + str(u2) + ", max_dif = " + str(max_dif))
    print("time elapsed: " + str(toc-tic) + " sec")
    """

    """
    # Finding random u1, u2 with min max_dif value
    random.seed(3)
    
    trials = 100
    min_dif = 50
    min_dif_list = list()
    for i in range(trials):
        for j in range(trials):
            u1 = (random.random(), random.random())
            u2 = (random.random(), random.random())
            print(str(u1) + ", " + str(u2))
            grid = make_grid(v, pt, GRID_SIZE, u1, u2, print_grid=False)
            max_dif, max_coord = find_max_dif(7,grid)
            print("max_dif = " + str(max_dif) + ", at " + str(max_coord))
            if max_dif < min_dif:
                min_dif = max_dif
                min_dif_list = [(u1,u2)]
            elif max_dif == min_dif:
                min_dif_list.append((u1,u2))
    
    print("min_dif = " + str(min_dif))
    print("min_dif_list: " + str(min_dif_list))
    
    """
    
    """
    # min_dif = 5
    min_dif_list = [((0.1369585656765282, 0.016999404787337746), (0.4391535837866679, 0.8587283739534641)), 
                   ((0.8421385012171291, 0.8272277649876393), (0.8554876974387439, 0.4254061697343674)), 
                   ((0.6017875457598535, 0.10854049519642694), (0.4343211044932448, 0.7550971544323263)), 
                   ((0.05464147301595168, 0.8340091362353484), (0.7091983960546916, 0.855960876965961)), 
                   ((0.009601617940946827, 0.2876408268945586), (0.14390753070834938, 0.5662824564679872)), 
                   ((0.5697351920636861, 0.28956470378753507), (0.7293022302825091, 0.061654116242531254)), 
                   ((0.12899564371330774, 0.022282518214378766), (0.07011812618338864, 0.7130289886950659))]
    

    min_min_dif_list = list()
    min_dif = 50
    for tup in min_dif_list:
        u1 = tup[0]
        u2 = tup[1]
        grid = make_grid(v, pt, GRID_SIZE, u1, u2, print_grid=False)
        max_dif, max_coord = find_max_dif(7,grid)
        print(str(u1) + ", " + str(u2))
        print("max_dif = " + str(max_dif) + ", at " + str(max_coord))
        if max_dif < min_dif:
            min_dif = max_dif
            min_min_dif_list = [(u1,u2)]
        elif max_dif == min_dif:
            min_min_dif_list.append((u1,u2))
            
    print("min_dif = " + str(min_dif))
    print("min_min_dif_list: " + str(min_min_dif_list))
    """ 
    
    """
    u1 = (0.11780364045386771, 0.9687294700524799) 
    u2 = (0.5714476731089351, 0.7153734688938803)
    grid = make_grid(v, pt, GRID_SIZE, u1, u2, print_grid=False)
    max_dif, max_coord = find_max_dif(7,grid)
    print(str(u1) + ", " + str(u2))
    print("max_dif = " + str(max_dif) + ", at " + str(max_coord))
    
    """
        